In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install jieba
!pip install nltk
!pip install transformers
!pip install sentencepiece
!pip install stanza
!pip install -U synonyms
!python -c "import synonyms"

In [ ]:
import spacy
from jieba import posseg
from nltk.corpus import wordnet 
import jieba
import nltk
import stanza
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import BertTokenizer, BertModel
import re
import synonyms
import torch
from torch.nn.functional import cosine_similarity

nltk.download('wordnet')
nlp_en = spacy.load("en_core_web_sm")


model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer_zh = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="zh_CN")
tokenizer_en = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")


In [ ]:
class Modtrans:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path


    def without_transp(self, en_pos = 'ADJ', zh_pos = 'a'):  # adj,  verb
        e = 151793

        for en,zh in zip(self.en_li[151793:], self.zh_li[151793:]):
            # en_verb_li = [tok_en.text for tok_en in nlp_en(en) if tok_en.pos_ == 'VERB' and tok_en.text.endswith("ing") and 'go' not in en and 'get' not in en and 'take' not in en]
            en_verb_li = [tok_en.text for tok_en in nlp_en(en) if tok_en.pos_ == en_pos]
            en_all_li = [tok_en.text for tok_en in nlp_en(en)]
            print(e)
            e += 1
            
            if en_verb_li != []:
                zh_verb_li = []
                for zh_tok,label in posseg.cut(zh):
                    if label == zh_pos:
                        zh_verb_li.append(zh_tok)
                
                if zh_verb_li != []:
                    en2zh_verb_li = []
                    for en_verb in en_verb_li:
                        en_text_trans = self.en2zh(en_verb)
                        en2zh_verb_li.append(en_text_trans)
                    tup_li = []
                    for i, en2zh_verb in enumerate(en2zh_verb_li):
                        for zh_verb in zh_verb_li:
                            if len(en2zh_verb.split()) == 1 and len(zh_verb.split()) == 1:
                                zh_text_trans =self.zh2en(zh_verb)



                                # if zh_text_trans in 
                                r = synonyms.compare(en2zh_verb,zh_verb, seg=False)
                                if r > 0.5:
                                    break
                                elif r >= 0.2 and r <= 0.5:
                                    if len(list(set(en2zh_verb).intersection(set(zh)))) == 0:
                                        tup = (en_verb_li[i],en2zh_verb,zh_verb,r)
                                        tup_li.append(tup)
                                    

                    for t in list(set(tup_li)):
                        print(en)
                        print(zh)                      
                        print(t)
                        print('--------')
                        with open(self.save_path, 'a+') as w:
                            w.write(en+'\n'+zh+'\n'+t[0]+':'+t[1]+':'+t[2]+'\n'+'\n')

    def with_transp(self, en_pos = 'ADJ', zh_pos = 'a'):  # adj,  verb
        e = 1

        for en,zh in zip(self.en_li, self.zh_li):
            # en_verb_li = [tok_en.text for tok_en in nlp_en(en) if tok_en.pos_ == 'VERB' and tok_en.text.endswith("ing") and 'go' not in en and 'get' not in en and 'take' not in en]
            en_verb_li = [tok_en.text for tok_en in nlp_en(en) if tok_en.pos_ == en_pos]
            en_all_li = [tok_en.text for tok_en in nlp_en(en)]
            print(e)
            e += 1
            if en_verb_li != []:
                zh_verb_li = []
                for zh_tok,label in posseg.cut(zh):
                    if label == zh_pos:
                        zh_verb_li.append(zh_tok)
                
                if zh_verb_li != []:
                    en2zh_verb_li = []
                    for en_verb in en_verb_li:
                        en_text_trans = self.en2zh(en_verb)
                        en2zh_verb_li.append(en_text_trans)
                    tup_li = []
                    for i, en2zh_verb in enumerate(en2zh_verb_li):
                        for zh_verb in zh_verb_li:
                            if len(en2zh_verb.split()) == 1 and len(zh_verb.split()) == 1:
                                zh_text_trans =self.zh2en(zh_verb)



                                # if zh_text_trans in 
                                r = synonyms.compare(en2zh_verb,zh_verb, seg=False)
                                if r > 0.4:
                                    break
                                elif r >= 0.2 and r <= 0.5:
                                    if len(list(set(en2zh_verb).intersection(set(zh)))) == 0:
                                        tup = (en_verb_li[i],en2zh_verb,zh_verb,r)
                                        tup_li.append(tup)
                                    

                    for t in list(set(tup_li)):
                        # print(en)
                        # print(zh)                      
                        # print(t)
                        # print('--------')
                        with open(self.save_path, 'a+') as w:
                            w.write(en+'\n'+zh+'\n'+t[0]+':'+t[1]+':'+t[2]+'\n'+'\n')


    def en2zh(self,word):
        en_token = tokenizer_en(word, return_tensors="pt")
        generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer_en.lang_code_to_id["zh_CN"])
        en_text_trans = tokenizer_en.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return en_text_trans
    



    def zh2en(self,word):
        en_token = tokenizer_zh(word, return_tensors="pt")
        generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer_zh.lang_code_to_id["en_XX"])
        zh_text_trans = tokenizer_zh.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return zh_text_trans



    def en_zh_similarity(self,word):
        inputs = tokenizer(word, return_tensors="pt")
        outputs = model(**inputs)
        embedding = outputs.encoder_last_hidden_state.mean(dim=1)
        return embedding



    def easy_way_mod(self, en_pos = 'ADJ', zh_pos = 'a'):
        for en,zh in zip(self.en_li, self.zh_li):
            en_verb_li = [tok_en.text for tok_en in nlp_en(en) if tok_en.pos_ == en_pos]
            if en_verb_li != []:
                zh_verb_li = []
                for zh_tok,label in posseg.cut(zh):
                    if label == zh_pos:
                        zh_verb_li.append(zh_tok)


                if zh_verb_li != []:
                    tup_li = []
                    for en_verb in en_verb_li:
                        for zh_verb in zh_verb_li:
                            if len(zh_verb.split()) == 1 and len(zh_verb.split()) == 1:
                                emb_en = self.en_zh_similarity(en_verb)
                                emb_zh = self.en_zh_similarity(zh_verb)
                                similarity = cosine_similarity(emb_en, emb_zh)
                                tup = (en_verb,zh_verb,similarity.item())
                                print(en)
                                print(zh)
                                print(tup)



                  



        
            
spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/Education/Bi-Education.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature67_mod_mod+transpo/mod_slight_mean_change_adj_new.txt'             
        
ptcl = Modtrans(spath, save_path)

ptcl.without_transp()


